<a href="https://colab.research.google.com/github/salmonin-o3o/ESAA_study/blob/main/250317_markdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

핸즈온

## 7.5 부스팅
부스팅은 약한 학습기를 여러 개 연결하여 강한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법을 말한다. ex) 에이다부스트, 그레이디언트 부스팅

### 7.5.1 에이다부스트
이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 높이는 것이다. 이것이 에이다부스트에서 사용하는 방식이다.

사이킷런은 SAMME라는 에이다부스트의 다중 클래스 버전을 사용한다. 클래스가 두 개뿐일 때는 SAMME가 에이다부스트와 동일하다.

다음 코드는 사이킷런의 AdaBoostClassifier를 사용하여 200개의 아주 얕은 결정 트리를 기반으로 하는 에이드부스트 분류기를 훈련시킨다.

In [1]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

# 5장에서 소개한 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X, y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그레이디언트 부스팅
그레이디언트 부스팅은 에이다부스트처럼 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킨다.

그레이디언트 트리 부스팅 또는 그레이디언트 부스티드 회귀트리를 연습해보자.

In [3]:
from sklearn.tree import DecisionTreeRegressor
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

이제 첫번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor를 훈련시킨다.

In [4]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

계속한다.

In [5]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

이제 모든 트리의 예측을 더한다.

In [6]:
# y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

트리가 앙상블에 추가될수록 앙상블의 예측이 점차 좋아진다. 또한, 사이킷런의 GradientBoostingRegressor를 사용하면 GBRT 앙상블을 간단하게 훈련시킬 수 있다.

In [7]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

learning_rate 매개변수가 각 트리의 기여 정도를 조절한다. 낮게 설정하면 학습시키기 위해 많은 트리가 필요하지만 예측 성능은 좋아진다. 이는 축소라고 부르는 규제 방법이다.(하지만 트리가 너무 많아도 과대적합이 일어난다.)

최적의 트리 수를 찾기 위해서는 조기 종료 기법을 사용할 수 있다. staged_predict()를 사용해보자.

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt= GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=81)

실제로 훈련을 중지하는 방법으로 조기 종료를 구현할 수도 있다. warm_start=True로 설정하면 사이킷런이 fit()이 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 해준다. 다음 코드는 연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춘다.

In [9]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
          break # 조기 종료

각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있는 subsample 매개변수도 있다. 편향이 높아지는 대신 분산이 낮아지는 효과가 있다. 또한 훈련 속도를 상당히 높인다. 이런 기법을 확률적 그레이디언트 부스팅이라고 한다.

최적화된 그레이디언트 부스팅 구현으로 XGBoost 파이썬 라이브러리가 유명하다. 이 패키지의 목표는 매우 빠른 속도, 확장성, 이식성이다.

In [11]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

자동 조기 종료와 같은 기능들이 있다.

In [13]:
xgb_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)])
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.36054
[1]	validation_0-rmse:0.26008
[2]	validation_0-rmse:0.18782
[3]	validation_0-rmse:0.13579
[4]	validation_0-rmse:0.09830
[5]	validation_0-rmse:0.07124
[6]	validation_0-rmse:0.05174
[7]	validation_0-rmse:0.03740
[8]	validation_0-rmse:0.02708
[9]	validation_0-rmse:0.02252
[10]	validation_0-rmse:0.01760
[11]	validation_0-rmse:0.01458
[12]	validation_0-rmse:0.01252
[13]	validation_0-rmse:0.01124
[14]	validation_0-rmse:0.00992
[15]	validation_0-rmse:0.00901
[16]	validation_0-rmse:0.00871
[17]	validation_0-rmse:0.00837
[18]	validation_0-rmse:0.00816
[19]	validation_0-rmse:0.00787
[20]	validation_0-rmse:0.00782
[21]	validation_0-rmse:0.00785
[22]	validation_0-rmse:0.00783
[23]	validation_0-rmse:0.00783
[24]	validation_0-rmse:0.00769
[25]	validation_0-rmse:0.00758
[26]	validation_0-rmse:0.00756
[27]	validation_0-rmse:0.00754
[28]	validation_0-rmse:0.00754
[29]	validation_0-rmse:0.00754
[30]	validation_0-rmse:0.00754
[31]	validation_0-rmse:0.00753
[32]	validation_0-

## 7.6 스태킹
스태킹은 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수(voting 같은)를 사용하는 대신 취합하는 모델을 훈련시키는 앙상블 모델이다.

마지막 예측기는 블렌더 또는 메타 학습기라고 한다.

블렌더를 학습시키는 일반적인 방법은 홀드 아웃 세트를 사용하는 것이다.

안타깝게도 사이킷런은 스태킹을 직접 지원하지 않는다.